In [1]:
import Covid19_model as Cm
import time
from read_training import *
import pickle
from Reinforce_learn import *
import pandas as pd
from Offline_test import *

In [2]:
"""model parameters"""
D = 1e2
n_site = 16
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))/pop
init_state = init_state.tolist()
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
init_state.append([1])
init_state.append([0]*n_site)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.7e6
path = 'result/transferRL_' + str(round(n_site)) + 'high'

In [3]:
"""system parameters"""
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
env = Cm.Env_model(init_state, deter_para, in_deter_para, seed=0)
env.set_para_truth(para_truth)
agent = read_training(path, n_state, n_action, env, 54, 54)


model have been loaded


In [5]:
tic = time.time()
action_store = []
state = env.init_state.copy()
observation = state[:6] * pop
state_buffer = np.zeros([T, 6])
new_buffer = np.zeros([T, 4])
budget, score, ni, ne = env.B, 0, 0, 0 
done = False
n_select = 5
epd = 1
p = np.zeros((1, n_site))
action_buffer = []
reward_buffer = []
index = np.array(range(n_select))
actor = agent[0]
start_time = 6
for t in range(T):
    p_set = env.para_sampling(100)
    if t > start_time: 
        agent, select_id = train_online(agent, env, index, epd, p_set)
        actor = agent[select_id]
    state_nn = state[0]+state[1]+state[2]+state[3]+state[4]+state[5]+state[6]+state[7]
    if t > start_time:
        act = actor.act_target(np.array(state_nn))[0]
        action = np.reshape(act, (5, n_site))   
    else:
        action = np.zeros((5, n_site))
    next_state, obs, reward, cost, new_i, new_e, ck, done = env.step_online(t, state, observation,
                                                                             action, p_set,
                                                                             n_select, 100)
    score += reward
    ni += new_i
    ne += new_e
    if not done and t > start_time: 
        env.space_update(ck, p_set, n_select)
        index = ck.argsort()[0][:n_select]
    action_store.append(action)
    action = np.reshape(action, (5, n_site))
    state = state[:6] * env.pop
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]), 
                  budget, score))
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]), 
                  np.sum(state[4]), np.sum(state[5]), budget, score))
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    print(action)
    budget -= cost
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time()-tic)

 Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2700000.00, reward:-229.27
 Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2700000.00, reward:-229.27
 Time: 0 | new_e:52.9, cum_e:52.9, new_i:47.3, cum_i:47.3
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 1 | S: 25495799.8, E: 147.8, A: 0.0, I: 47.3, D: 0.0, R: 5.1, Budget: 2700000.00, reward:-554.35
 Time: 1 | S: 25495797.0, E: 144.0, A: 0.0, I: 47.3, D: 0.0, R: 4.0, Budget: 2700000.00, reward:-554.35
 Time: 1 | new_e:74.3, cum_e:127.2, new_i:65.4, cum_i:112.6
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 

[0.15620516246224594, 0.15682342954204143]
[0.812362022366947, 0.8134873492599886]
[0.09822124804868297, 0.09825627118354618]
[0.0417233065074914, 0.04180742582787621]
 Time: 10 | S: 25259782.3, E: 264.3, A: 1210.9, I: 1752.7, D: 3.0, R: 232986.7, Budget: 2552764.60, reward:-10874.89
 Time: 10 | S: 25260014.0, E: 254.0, A: 1210.9, I: 1752.7, D: 3.0, R: 232967.0, Budget: 2552764.60, reward:-10874.89
 Time: 10 | new_e:123.3, cum_e:2462.9, new_i:240.3, cum_i:2485.5
[[1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999988e-01 1.0000000e+00
  9.4278437e-01 1.0000000e+00 9.6938474e-04 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.6998017e-08 1.0000000e+00 9.9999809e-01
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 9.9999988e-01 9.9999988e-01 7.5903185e-02 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  9.9581194e-01]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999988

[0.1564309424202712, 0.15643244279498736]
[0.8134611086618689, 0.8134613106734907]
[0.09825269267455194, 0.09825269303215076]
[0.04180293473587777, 0.04180293608918308]
 Time: 16 | S: 24814717.5, E: 51.2, A: 735.9, I: 1557.4, D: 5.0, R: 678932.9, Budget: 2275511.47, reward:-12286.62
 Time: 16 | S: 24814932.0, E: 54.0, A: 735.9, I: 1557.4, D: 5.0, R: 678892.0, Budget: 2275511.47, reward:-12286.62
 Time: 16 | new_e:23.5, cum_e:2763.8, new_i:100.0, cum_i:3376.9
[[1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  8.3484304e-01 1.0000000e+00 2.9103721e-05 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 4.2526906e-11 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9996698e-01 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00

[0.15643149928131359, 0.1564315039669565]
[0.8134613073022563, 0.8134613073164002]
[0.09825269298707993, 0.09825269298708843]
[0.04180293596589819, 0.04180293596591178]
 Time: 22 | S: 24403167.8, E: 17.0, A: 330.1, I: 997.4, D: 5.9, R: 1091481.7, Budget: 2021366.89, reward:-12655.51
 Time: 22 | S: 24403349.0, E: 25.0, A: 330.1, I: 997.4, D: 5.9, R: 1091428.0, Budget: 2021366.89, reward:-12655.51
 Time: 22 | new_e:7.7, cum_e:2836.5, new_i:42.3, cum_i:3749.3
[[1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.8520309e-01 1.0000000e+00 1.0361132e-06 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 8.5752662e-11 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999988e-01 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 

[0.15643150074913123, 0.15643150099119818]
[0.8134613073026323, 0.8134613073026343]
[0.09825269298708735, 0.09825269298708735]
[0.04180293596591146, 0.041802935965911475]
 Time: 28 | S: 24022451.5, E: 3.5, A: 141.4, I: 571.2, D: 6.5, R: 1472825.9, Budget: 1754183.28, reward:-12779.12
 Time: 28 | S: 24022600.0, E: 3.0, A: 141.4, I: 571.2, D: 6.5, R: 1472766.0, Budget: 1754183.28, reward:-12779.12
 Time: 28 | new_e:1.6, cum_e:2857.6, new_i:16.6, cum_i:3902.8
[[1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999988e-01 1.0000000e+00
  8.0268523e-03 1.0000000e+00 1.0356153e-08 1.0000000e+00 1.0000000e+00
  1.0000000e+00 9.9999988e-01 1.6775931e-01 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999988e-01 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 

...

[0.1564315008734492, 0.15643150087345062]
[0.8134613073026334, 0.8134613073026341]
[0.09825269298708735, 0.09825269298708735]
[0.041802935965911475, 0.041802935965911475]
 Time: 62 | S: 22224018.9, E: 0.0, A: 0.6, I: 10.6, D: 7.2, R: 3271962.7, Budget: 373526.68, reward:-12838.69
 Time: 62 | S: 22224171.0, E: 0.0, A: 0.6, I: 10.6, D: 7.2, R: 3271918.0, Budget: 373526.68, reward:-12838.69
 Time: 62 | new_e:0.0, cum_e:2863.2, new_i:0.1, cum_i:3997.4
[[1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  5.5948912e-10 1.0000000e+00 8.6435551e-07 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 2.8673213e-04 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.000000

[0.15643150087344956, 0.1564315008734497]
[0.8134613073026334, 0.8134613073026341]
[0.09825269298708735, 0.09825269298708735]
[0.041802935965911475, 0.041802935965911475]
 Time: 68 | S: 21959798.6, E: 0.0, A: 0.2, I: 4.9, D: 7.2, R: 3536189.0, Budget: 157964.66, reward:-12839.13
 Time: 68 | S: 21959947.0, E: 0.0, A: 0.2, I: 4.9, D: 7.2, R: 3536147.0, Budget: 157964.66, reward:-12839.13
 Time: 68 | new_e:0.0, cum_e:2863.2, new_i:0.0, cum_i:3997.7
[[1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  2.0914237e-10 1.0000000e+00 4.6927935e-06 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 2.4614646e-04 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e

 Time: 74 | S: 21832420.2, E: 0.7, A: 0.1, I: 3.5, D: 7.2, R: 3663568.3, Budget: 384.37, reward:-12845.74
 Time: 74 | S: 21832566.0, E: 0.0, A: 0.1, I: 3.5, D: 7.2, R: 3663530.0, Budget: 384.37, reward:-12845.74
 Time: 74 | new_e:0.6, cum_e:2864.6, new_i:0.5, cum_i:3998.8
[[1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  3.9869393e-12 1.0000000e+00 5.4033757e-07 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 6.4028939e-07 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+0

 Time: 80 | S: 21952839.6, E: 7.8, A: 0.0, I: 11.6, D: 7.2, R: 3543133.7, Budget: 384.37, reward:-12905.28
 Time: 80 | S: 21952994.0, E: 0.0, A: 0.0, I: 11.6, D: 7.2, R: 3543099.0, Budget: 384.37, reward:-12905.28
 Time: 80 | new_e:4.2, cum_e:2877.7, new_i:3.6, cum_i:4010.1
[[1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  2.7012895e-10 1.0000000e+00 9.9194604e-01 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 8.0806356e-05 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999917e-01 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e

 Time: 86 | S: 22069212.0, E: 51.9, A: 0.0, I: 65.8, D: 7.4, R: 3426662.8, Budget: 384.37, reward:-13287.93
 Time: 86 | S: 22069444.0, E: 0.0, A: 0.0, I: 65.8, D: 7.4, R: 3426624.0, Budget: 384.37, reward:-13287.93
 Time: 86 | new_e:26.9, cum_e:2962.7, new_i:23.5, cum_i:4084.3
[[1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.8366770e-11 1.0000000e+00 1.8223490e-07 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 4.9235136e-06 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00]
 [1.0000000e+00 1.00000

In [6]:
result_state = pd.DataFrame(state_buffer)
result_state.to_csv(r'TL_high_state_16.csv')
result_new = pd.DataFrame(new_buffer)
result_new.to_csv(r'TL_high_new_16.csv')

In [10]:
import pickle
pickle.dump(action_store, open("TL_high_action.pth", 'wb'))

In [8]:
(time.time()-tic) / ( T - start_time + 1)

66.01981692875133

In [9]:
4219.7 + 3117.7

7337.4